A Fórmula Mágica é um modelo de fator que classifica as ações por dois fatores:

1. O valor de uma empresa em relação aos seus lucros determina quão “barato” é o preço de mercado de uma ação. Greenblatt define “Barato” como o valor de uma empresa em relação aos seus lucros. Na maioria das vezes, podemos ver isto representado pelo P/L, Greenblatt prefere olhar para EV/EBIT. Isto permite que empresas com diferentes estruturas de endividamento e impostos sejam comparadas mais facilmente. O EV/EBIT compara o Valor da Empresa (Enterprise Value, EV = Capitalização de mercado + Dívida total - Caixa e equivalentes de caixa) ao seu Lucro Antes de Juros e Impostos (Earnings Before Interest and Taxes, EBIT), o EV leva em conta o valor de mercado total da empresa, adicionando as dívidas e subtraindo o caixa, oferecendo assim, uma visão mais abrangente do valor da empresa do que simplesmente seu valor de mercado. O EBIT é uma medida da capacidade de geração de lucro operacional da empresa, excluindo os efeitos das decisões de estrutura de capital e impostos. Um EV/EBIT menor sugere que o preço da ação da empresa pode estar baixo em comparação com a quantidade de lucro operacional que está gerando, indicando que a empresa está subavaliada pelo mercado. No entanto, é importante analisar o EV/EBIT dentro do contexto de cada setor, já que diferentes indústrias têm padrões de avaliação distintos.

2. O retorno sobre o capital determina o quão “boa” é uma empresa. “Bom” é representado pelo ROIC, Greenblatt quantifica a quantidade de capital tangível necessária para operar um negócio e quanto dinheiro cada unidade de capital aplicado irá render. O ROIC é um dos indicadores financeiros mais importantes para analisar a eficiência operacional de uma empresa. Para calcular o ROIC, divide-se o lucro operacional líquido de impostos pelo capital investido total, que inclui dívidas e patrimônio líquido. Um ROIC alto é geralmente interpretado como um sinal de que a empresa está utilizando seu capital de maneira eficaz para gerar lucros.

A fim de aderir mais fielmente aos princípios de Greenblatt, empresas pertencentes aos setores de utilidades públicas, bancos e seguradoras são excluídas da seleção.



In [1]:
earning_yield = "EV/EBIT"
return_on_capital = "ROIC"

In [2]:
import warnings
warnings.filterwarnings('ignore')
# from datetime import datetime, date
import numpy as np
import pandas as pd
# import yfinance as yf

def pct_to_float(number):
    """Convert string to float, remove % char and set decimal point to '.'."""
    return float(number.strip("%").replace(".", "").replace(",", "."))

dados do site https://www.fundamentus.com.br/resultado.php

In [3]:
import requests
header = {
  "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36",
  "X-Requested-With": "XMLHttpRequest"
}
url = 'https://www.fundamentus.com.br/resultado.php'
#junta com a requests
r = requests.get(url, headers=header)
# read_html do pandas põe a tabela num dataframe
funda = pd.read_html(r.text, index_col="Papel",
                     decimal=',', thousands='.',encoding='ISO-8859-1', 
                     converters={'ROE': pct_to_float,
                                 'ROIC': pct_to_float,
                                 'Div.Yield':pct_to_float,
                                 'Mrg Ebit':pct_to_float,
                                 'Mrg. Líq.':pct_to_float,
                                 'Cresc. Rec.5a':pct_to_float,
                                 },
)[0]
dfunds = pd.DataFrame(funda)


In [4]:
dfunds.columns

Index(['Cotação', 'P/L', 'P/VP', 'PSR', 'Div.Yield', 'P/Ativo', 'P/Cap.Giro',
       'P/EBIT', 'P/Ativ Circ.Liq', 'EV/EBIT', 'EV/EBITDA', 'Mrg Ebit',
       'Mrg. Líq.', 'Liq. Corr.', 'ROIC', 'ROE', 'Liq.2meses', 'Patrim. Líq',
       'Dív.Brut/ Patrim.', 'Cresc. Rec.5a'],
      dtype='object')

filtros e ranking

In [5]:
funds = dfunds.copy()

funds =  funds[funds[earning_yield] > 0]
funds =  funds[funds[return_on_capital] > 0]

funds =  funds[funds['Liq.2meses'] > 300000]    #Volume diário médio negociado nos últimos 2 meses
funds =  funds[funds['P/L'] > 0]                # sem prejuizo atual
funds =  funds[(funds['P/L'] < 60)]           # não estar excessivamente cara
funds =  funds[funds['Dív.Brut/ Patrim.'] < 4]  # endividamento sob controle
funds =  funds[funds['Cresc. Rec.5a'] > 0]    # crescimento nos ultimos 5 anos


""" magic formula rank."""
funds["Rank_earnings_yield"]   = funds[earning_yield].rank(ascending=True, method="min")
funds["Rank_return_on_capital"]= funds[return_on_capital].rank(ascending=False, method="min")
funds["Rank_Final"] = (funds["Rank_earnings_yield"] + funds["Rank_return_on_capital"])
funds.sort_values(by="Rank_Final", ascending=True, inplace=True)
funds.reset_index(inplace=True)
funds.index = funds.index + 1

In [6]:
from IPython.display import display, HTML
display(HTML(funds.head(30).to_html(index=False)))

Papel,Cotação,P/L,P/VP,PSR,Div.Yield,P/Ativo,P/Cap.Giro,P/EBIT,P/Ativ Circ.Liq,EV/EBIT,EV/EBITDA,Mrg Ebit,Mrg. Líq.,Liq. Corr.,ROIC,ROE,Liq.2meses,Patrim. Líq,Dív.Brut/ Patrim.,Cresc. Rec.5a,Rank_earnings_yield,Rank_return_on_capital,Rank_Final
PSSA3,31.28,8.92,1.63,0.624,4.51,0.457,6.53,0.76,-3.83,0.31,0.30,82.58,7.15,1.13,83.01,18.25,52077800.0,1.241530e+10,0.00,16.93,1.0,1.0,2.0
SHOW3,2.90,3.41,1.10,0.279,0.00,0.388,3.75,2.02,-6.00,1.06,1.01,13.82,8.30,1.22,46.38,32.41,579888.0,1.770350e+08,0.31,38.53,2.0,2.0,4.0
CMIN3,5.30,8.15,2.72,1.540,14.66,0.979,4.20,4.26,-4.84,4.02,3.49,36.17,18.90,2.14,37.86,33.36,44766400.0,1.069870e+10,0.76,10.09,11.0,5.0,16.0
WIZC3,7.24,8.18,2.40,1.032,6.15,0.455,-52.23,2.14,-1.42,2.25,1.84,48.14,21.04,0.96,25.65,29.40,2474190.0,4.816420e+08,0.74,12.32,5.0,16.0,21.0
PLPL3,14.00,10.65,4.63,1.380,4.75,1.402,2.66,4.89,8.44,4.82,4.69,28.24,13.36,2.63,42.81,43.52,9575090.0,6.171470e+08,0.63,30.46,19.0,3.0,22.0
PETR4,37.36,3.91,1.28,0.952,19.40,0.464,-71.16,2.06,-0.95,3.02,2.29,46.25,24.45,0.96,24.87,32.75,1769720000.0,3.804410e+11,0.80,21.06,6.0,17.0,23.0
PETR3,38.28,4.01,1.31,0.975,18.93,0.475,-72.91,2.11,-0.98,3.07,2.32,46.25,24.45,0.96,24.87,32.75,582275000.0,3.804410e+11,0.80,21.06,7.0,17.0,24.0
CAMB3,11.27,6.63,2.05,1.038,2.06,1.420,5.89,4.74,10.70,4.65,4.15,21.89,15.65,2.22,33.30,30.87,805084.0,2.328300e+08,0.04,28.50,16.0,8.0,24.0
VLID3,18.49,7.09,1.17,0.671,5.18,0.594,2.11,3.27,28.51,3.85,3.06,20.53,9.34,2.30,23.84,16.43,7954730.0,1.298460e+09,0.55,2.03,10.0,19.0,29.0
VALE3,60.83,6.91,1.45,1.327,11.49,0.606,14.10,3.24,-1.65,3.82,3.24,40.99,19.49,1.28,20.67,20.91,1712210000.0,1.909650e+11,0.35,7.85,9.0,25.0,34.0


retira banco, seguradora, utilidade publica (agua e luz - [util-b3](https://www.b3.com.br/pt_br/market-data-e-indices/indices/indices-de-segmentos-e-setoriais/indice-utilidade-publica-util-composicao-da-carteira.htm))


In [7]:
def setor(setor:int):
    url = f'http://www.fundamentus.com.br/resultado.php?setor={setor}'
    hdr = {'User-agent': 'Mozilla/5.0 (Windows; U; Windows NT 6.1; rv:2.2) Gecko/20110201' ,
           'Accept': 'text/html, text/plain, text/css, text/sgml, */*;q=0.01' ,
           'Accept-Encoding': 'gzip, deflate' ,
           }
    content = requests.get(url, headers=hdr)
    df = pd.read_html(content.text, decimal=",", thousands='.')[0]
    return list(df['Papel'])


In [8]:
bancos = setor(20)
print(*bancos)
seguros = setor(31)
print(*seguros)
util_agua = setor(2)
util_luz = setor(14)
util=util_agua+util_luz
print(*util)


BAZA3 BNBR3 BBAS3 PINE4 BMEB3 BMEB4 BRSR3 BRSR6 ABCB4 BEES3 BRSR5 ITSA3 ITSA4 BEES4 ITUB3 BGIP4 BBDC3 PINE3 BMGB4 BBDC4 BGIP3 ITUB4 DMFN3 BMIN4 BPAC5 BRBI11 SANB3 SANB11 SANB4 BMIN3 BPAC11 BPAN4 BPAC3 BSLI4 BSLI3 CRIV4 CRIV3 RPAD5 RPAD3 MERC4 RPAD6 BRIV4 BRIV3
APER3 IRBR3 WIZC3 BBSE3 PSSA3 CXSE3 BRGE6 BRGE7 BRGE8 BRGE11 BRGE5 BRGE12 BRGE3
AMBP3 SAPR3 SAPR11 SAPR4 CSMG3 SBSP3 ORVR3 AURE3 RNEW4 RNEW11 RNEW3 CBEE3 GPAR3 CLSC3 CLSC4 CMIG4 CEEB5 NEOE3 CMIG3 CEEB3 TRPL4 EQMA3B CSRN5 CSRN3 CSRN6 GEPA3 GEPA4 CPFE3 CEBR5 EQPA5 EQPA7 LIGT3 TRPL3 EQPA3 CEBR6 CEBR3 REDE3 EQPA6 TAEE3 TAEE11 COCE5 TAEE4 EKTR4 EGIE3 ENGI4 EKTR3 CPLE3 ENGI11 ENMT4 ENMT3 COCE3 CPLE6 ALUP4 ALUP11 ALUP3 AFLT3 EMAE4 EQTL3 ENGI3 ELET3 ELET6 CPLE5 LIPR3 AESB3 ELET5 ENEV3 SRNA3


In [9]:
funds = funds[(~funds['Papel'].isin(bancos))&(~funds['Papel'].isin(seguros))&(~funds['Papel'].isin(util))]
display(HTML(funds.head(30).to_html(index=False)))

Papel,Cotação,P/L,P/VP,PSR,Div.Yield,P/Ativo,P/Cap.Giro,P/EBIT,P/Ativ Circ.Liq,EV/EBIT,EV/EBITDA,Mrg Ebit,Mrg. Líq.,Liq. Corr.,ROIC,ROE,Liq.2meses,Patrim. Líq,Dív.Brut/ Patrim.,Cresc. Rec.5a,Rank_earnings_yield,Rank_return_on_capital,Rank_Final
SHOW3,2.90,3.41,1.10,0.279,0.00,0.388,3.75,2.02,-6.00,1.06,1.01,13.82,8.30,1.22,46.38,32.41,579888.0,1.770350e+08,0.31,38.53,2.0,2.0,4.0
CMIN3,5.30,8.15,2.72,1.540,14.66,0.979,4.20,4.26,-4.84,4.02,3.49,36.17,18.90,2.14,37.86,33.36,44766400.0,1.069870e+10,0.76,10.09,11.0,5.0,16.0
PLPL3,14.00,10.65,4.63,1.380,4.75,1.402,2.66,4.89,8.44,4.82,4.69,28.24,13.36,2.63,42.81,43.52,9575090.0,6.171470e+08,0.63,30.46,19.0,3.0,22.0
PETR4,37.36,3.91,1.28,0.952,19.40,0.464,-71.16,2.06,-0.95,3.02,2.29,46.25,24.45,0.96,24.87,32.75,1769720000.0,3.804410e+11,0.80,21.06,6.0,17.0,23.0
PETR3,38.28,4.01,1.31,0.975,18.93,0.475,-72.91,2.11,-0.98,3.07,2.32,46.25,24.45,0.96,24.87,32.75,582275000.0,3.804410e+11,0.80,21.06,7.0,17.0,24.0
CAMB3,11.27,6.63,2.05,1.038,2.06,1.420,5.89,4.74,10.70,4.65,4.15,21.89,15.65,2.22,33.30,30.87,805084.0,2.328300e+08,0.04,28.50,16.0,8.0,24.0
VLID3,18.49,7.09,1.17,0.671,5.18,0.594,2.11,3.27,28.51,3.85,3.06,20.53,9.34,2.30,23.84,16.43,7954730.0,1.298460e+09,0.55,2.03,10.0,19.0,29.0
VALE3,60.83,6.91,1.45,1.327,11.49,0.606,14.10,3.24,-1.65,3.82,3.24,40.99,19.49,1.28,20.67,20.91,1712210000.0,1.909650e+11,0.35,7.85,9.0,25.0,34.0
LEVE3,34.57,6.26,5.15,1.080,27.19,1.673,7.63,5.77,-21.77,6.44,5.72,18.71,17.23,1.58,38.76,82.19,18804500.0,9.103790e+08,0.98,17.89,39.0,4.0,43.0
UNIP3,62.51,8.25,2.68,1.326,7.21,1.015,3.70,6.34,-7.18,6.27,5.05,20.90,16.16,2.36,25.97,32.51,478043.0,2.422330e+09,0.85,17.10,37.0,13.0,50.0


retirar BDR

In [10]:
# Padrão a ser buscado XYZW3[2,3,4,5]
padrao = r'[A-Z]{4}3[2-5]'

# Filtrando as linhas onde o padrão ocorre na coluna 'Papel'
funds = funds[funds['Papel'].str.contains(padrao)]
funds

,Papel,Cotação,P/L,P/VP,PSR,Div.Yield,P/Ativo,P/Cap.Giro,P/EBIT,P/Ativ Circ.Liq,...,Liq. Corr.,ROIC,ROE,Liq.2meses,Patrim. Líq,Dív.Brut/ Patrim.,Cresc. Rec.5a,Rank_earnings_yield,Rank_return_on_capital,Rank_Final
28,AURA33,37.29,16.3,1.73,1.268,5.24,0.59,3.0,5.22,-2.37,...,1.92,17.58,10.62,1837650.0,1.524050e+09,1.06,21.36,34.0,35.0,69.0


A fórmula:
1. Estabeleça uma capitalização de mercado mínima.
2. Exclua ações de serviços públicos e financeiras  (ou seja, fundos mútuos, bancos e companhias de seguros).
3. Excluir empresas estrangeiras (ADRs)
4. Determine o rendimento dos lucros da empresa = EBIT/EV
5. Determine o retorno sobre o capital da empresa = EBIT/ (Ativo Fixo Líquido + Capital de Giro).
6. Classifique todas as empresas acima da capitalização de mercado escolhida pelo maior rendimento de lucros e maior retorno sobre o capital (classificado como porcentagens)
7. Invista em 20 a 30 empresas com melhor classificação, acumulando 2 a 3 posições por mês durante um período de 12 meses
8. Reequilibre o portfólio uma vez por ano, vendendo os perdedores uma semana antes do ano e os vencedores uma semana depois do ano.
9. Continuar por um período de longo prazo (5 a 10+ anos)

https://en.wikipedia.org/wiki/Magic_formula_investing

O retorno sobre o capital de Greenblatt difere de um valor típico de ROE ou ROIC. Dentro da Fórmula Mágica, o retorno sobre o capital de uma empresa é medido como EBIT/capital tangível empregado. Em outras palavras, estamos tentando encontrar os custos tangíveis para o negócio na geração dos lucros reportados dentro do período, onde o capital tangível empregado é definido mais precisamente como Capital Circulante Líquido mais Ativos Fixos Líquidos.

O capital de giro líquido é simplesmente o total do ativo circulante menos o passivo circulante, com um ajuste para remover dívidas com juros de curto prazo do passivo circulante e outro para remover o excesso de caixa. Greenblatt não oferece detalhes sobre como o excesso de caixa deve ser considerado, mas muitas vezes é calculado com base em uma porcentagem do caixa necessário em relação às vendas geradas em um período.